## Trying predicting None with other Products and F1 optimization approche to improve f1 score

<p>Idea is we gonne create two different models one that we have already created that predicts products and the other one is to predict None(we are gonne consider none as product and gonne predict it with other products)</p>
<p>Step 1 - First get the features for None model, I have selected subset of the features from the previous Model we had</p>
<p>Step 2 - Apply Gradient Boosting Model after choosing best parameters, then predict the Test data and save the probablities into a file</p>
<p>Step 3 - get the probablities for the Gradient Boosting model that we applied before.</p>
<p>Step 4 - So now we would be having probablities from two different model, now we need to merge these probablities and apply F1 optimization to this to choose best f1 score from given probablity values, that will give the top prodicts for a order with None as product included.</p>

In [9]:
import os
import numpy as np
import pandas as pd

In [54]:
path = "Data"
order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order':np.uint8,
                                                                                  'reordered': bool})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number':np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order':np.uint8,
                                                                                  'reordered': bool})


### why do we need to predict None ?

In [141]:
order = orders[orders['eval_set']=="train"]
merged = order.merge(order_train, on='order_id')
here = merged.groupby(by='order_id').agg({'reordered':'sum'})
number_none_orders = here[here['reordered']==0].shape[0]
print('Percentage of orders that are without any reorder in Train: ',(number_none_orders/order.shape[0])*100,"%")

Percentage of orders that are without any reorder in Train:  6.555952716658156 %


### Getting Train and Test Data for None Model

In [120]:
data_train = pd.read_csv('my_final/data.csv')[['user_id','order_id','NumberTimeProductsOrdered','unique_prod','user_orders_count','order_dow','order_hour_of_day','days_since_prior_order','user_reorder_ratio','order_dow_mean','order_hour_of_day_mean','days_since_prior_order_mean','Avergae_Basket_size']]

In [121]:
# Train Data
data_train1 = data_train.copy()
data_train1=data_train1.drop_duplicates()
data_train1 = data_train1.groupby('user_id').agg('mean')
data_train1['user_id']=data_train1.index
#reading labels
order = orders[orders['eval_set']=="train"]
merged = order.merge(order_train, on='order_id')
here = merged.groupby(by='order_id').agg({'reordered':'sum'})
here.loc[here['reordered']>0,'reordered']=1
here = here.rename(columns={'reordered':'is_none'})
ok = here['is_none'].values
okk = np.where(ok>0,0,1)
here['is_none']=okk
data_train1 = data_train1.merge(here, on='order_id')
labels_train = data_train1['is_none']

# filling NaN values as -1
data_train1 = data_train1.fillna(-1)


In [123]:
# Adding one more feature to train: none_order_rate

merged = orders.merge(order_prior, on='order_id')
merged = merged[merged['order_number']!=1]
here = merged.groupby(by='order_id').agg({'reordered':'sum'})
ok = merged[['order_id','user_id','eval_set']].merge(here,on='order_id', how='inner').drop_duplicates()
ok.loc[ok['reordered']>0,'reordered']=1
ok1 = ok.groupby('user_id').agg({'reordered':['sum','count']})
ok1.columns = ok1.columns.levels[1]
ok1['none_order_rate']= (ok1['count']-ok1['sum'])/ok1['count']

data_train1 = data_train1.merge(ok1[['none_order_rate']],on='user_id')

In [124]:
#Test data
data_test1 = pd.read_csv('my_final/test_data.csv')[['user_id','order_id','NumberTimeProductsOrdered','unique_prod','user_orders_count','order_dow','order_hour_of_day','days_since_prior_order','user_reorder_ratio','order_dow_mean','order_hour_of_day_mean','days_since_prior_order_mean','Avergae_Basket_size']]
data_test1=data_test1.drop_duplicates()
data_test1 = data_test1.groupby('user_id').agg('mean')

# filling NaN values as -1
data_test1 = data_test1.fillna(-1)

In [125]:
# Adding one more feature to test: none_order_rate

merged = orders.merge(order_prior, on='order_id')
merged = merged[merged['order_number']!=1]
here = merged.groupby(by='order_id').agg({'reordered':'sum'})
ok = merged[['order_id','user_id','eval_set']].merge(here,on='order_id', how='inner').drop_duplicates()
ok.loc[ok['reordered']>0,'reordered']=1
ok1 = ok.groupby('user_id').agg({'reordered':['sum','count']})
ok1.columns = ok1.columns.levels[1]
ok1['none_order_rate']= (ok1['count']-ok1['sum'])/ok1['count']

data_test1 = data_test1.merge(ok1[['none_order_rate']],on='user_id')
data_test1['user_id']= data_test1.index

In [128]:
# # Save data files
# data_train1.to_csv('my_final/train_none_Data.csv')
# data_test1.to_csv('my_final/test_none_Data.csv')

# #read Data files
# data_train1 = pd.read_csv('my_final/train_none_Data.csv')
# data_test1 = pd.read_csv('my_final/test_none_Data.csv')

## Gradient Boosting model for None

### hyper-parameter tuning for None model

In [38]:
# read datasets
train = data_train1.copy()
test=data_test1.copy()
order_id=test['order_id']
y_train = train["is_none"]

dtrain = xgb.DMatrix(train.drop(['user_id','order_id','is_none'], axis=1), y_train)
dtest = xgb.DMatrix(test.drop(['user_id','order_id'], axis=1))
min_logloss = float("Inf")

grid_param = [(eta,gamma,max_depth,subsample,colsample_bytree,lambdaa,alpha,objective) 
              for eta in (0.005,0.01) for gamma in (0.1,0.5) for max_depth in (4,6) for subsample in (0.7,0.8) for colsample_bytree in (0.8,0.9) for lambdaa in (1,5) for alpha in (0.5,1) for objective in ('reg:logistic','reg:squarederror')]


for eta,gamma,max_depth,subsample,colsample_bytree,lambdaa,alpha,objective in grid_param:
    
    xgb_params = {'eta':eta, 
              'gamma':gamma,
              'max_depth':max_depth,
              'subsample':subsample,
              'colsample_bytree':colsample_bytree, 
              'lambda':lambdaa, 
              'alpha':alpha, 
              'objective':objective}
    cv_results = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=3,
                   num_boost_round=500, # increase to have better results (~700)
                   early_stopping_rounds=50,
                    metrics={'logloss'},
                   show_stdv=False
                  )
     # Update best MAE
    mean_logloss = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print(eta,gamma,max_depth,subsample,colsample_bytree,lambdaa,alpha,objective)
    print("logloss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (eta,gamma,max_depth,subsample,colsample_bytree,lambdaa,alpha,objective)
print("Best params: ", best_params, min_logloss)

0.005 0.1 4 0.7 0.8 1 0.5 reg:logistic
logloss 0.211985 for 499 rounds
0.005 0.1 4 0.7 0.8 1 0.5 reg:squarederror
logloss 0.21062266666666665 for 499 rounds
0.005 0.1 4 0.7 0.8 1 1 reg:logistic
logloss 0.21202600000000002 for 499 rounds
0.005 0.1 4 0.7 0.8 1 1 reg:squarederror
logloss 0.21065366666666666 for 499 rounds
0.005 0.1 4 0.7 0.8 5 0.5 reg:logistic
logloss 0.2122383333333333 for 499 rounds
0.005 0.1 4 0.7 0.8 5 0.5 reg:squarederror
logloss 0.21065466666666666 for 499 rounds
0.005 0.1 4 0.7 0.8 5 1 reg:logistic
logloss 0.21229333333333333 for 499 rounds
0.005 0.1 4 0.7 0.8 5 1 reg:squarederror
logloss 0.21067866666666668 for 499 rounds
0.005 0.1 4 0.7 0.9 1 0.5 reg:logistic
logloss 0.21199400000000002 for 499 rounds
0.005 0.1 4 0.7 0.9 1 0.5 reg:squarederror
logloss 0.210615 for 499 rounds
0.005 0.1 4 0.7 0.9 1 1 reg:logistic
logloss 0.21203466666666668 for 499 rounds
0.005 0.1 4 0.7 0.9 1 1 reg:squarederror
logloss 0.21064366666666667 for 499 rounds
0.005 0.1 4 0.7 0.9 5 0.5 r

In [132]:
# we are gonne predict None for the orders and store the probablity values into a file 
import xgboost as xgb
# read datasets
train = data_train1.copy()
test=data_test1.copy()
order_id=test['order_id']
y_train = train["is_none"]
       
print('Shape train: {}\n Shape test: {}'.format(train.shape,test.shape))

dtrain = xgb.DMatrix(train.drop(['user_id','order_id','is_none'], axis=1), y_train)
dtest = xgb.DMatrix(test.drop(['user_id','order_id'], axis=1))

xgb_params = {'eta':0.01, 
              'gamma':0.1,
              'max_depth':6,
              'subsample':0.8,
              'colsample_bytree':0.8,
              'lambda':1,
              'alpha':1, 
              'objective':'reg:squarederror',
             'eval_metric':'logloss'}

watchlist= [(dtrain, "train")]
num_boost_rounds=1500
model = xgb.train(dict(xgb_params), dtrain, num_boost_round=num_boost_rounds, evals=watchlist, verbose_eval = 50,)
preds=model.predict(dtest)

out = pd.DataFrame({'order_id': order_id, 'none_pred': preds})
out.to_csv('my_final/None_Probablity.csv', index=False)

Shape train: (131209, 15)
 Shape test: (75000, 14)
[0]	train-logloss:0.68528
[50]	train-logloss:0.43004
[100]	train-logloss:0.31252
[150]	train-logloss:0.25284
[200]	train-logloss:0.22121
[250]	train-logloss:0.20408
[300]	train-logloss:0.19466
[350]	train-logloss:0.18928
[400]	train-logloss:0.18608
[450]	train-logloss:0.18403
[500]	train-logloss:0.18263
[550]	train-logloss:0.18160
[600]	train-logloss:0.18069
[650]	train-logloss:0.18014
[700]	train-logloss:0.17967
[750]	train-logloss:0.17900
[800]	train-logloss:0.17857
[850]	train-logloss:0.17796
[900]	train-logloss:0.17729
[950]	train-logloss:0.17666
[1000]	train-logloss:0.17605
[1050]	train-logloss:0.17546
[1100]	train-logloss:0.17461
[1150]	train-logloss:0.17401
[1200]	train-logloss:0.17347
[1250]	train-logloss:0.17288
[1300]	train-logloss:0.17231
[1350]	train-logloss:0.17196
[1400]	train-logloss:0.17124
[1450]	train-logloss:0.17074
[1499]	train-logloss:0.17021


## Gradient Boosting model for predicting Products
<p> like we did in previous files</p

In [134]:
# Reading all the features
data_train1 = pd.read_csv('my_final/data.csv')
labels_train = data_train1['reordered']

# filling NaN values as -1
data_train1 = data_train1.fillna(-1)

data_test1 = pd.read_csv('my_final/test_data.csv')

# filling NaN values as -1
data_test1 = data_test1.fillna(-1)

In [135]:
data_train = data_train1.drop(['Unnamed: 0', 'order_id','eval_set_x','eval_set_y','reordered','product_name'],axis=1)
data_test = data_test1.drop(['Unnamed: 0', 'order_id','eval_set_x','eval_set_y','reordered','product_name'],axis=1)

X_train = data_train.copy()
y_train = labels_train.copy()
X_Test = data_test.copy()

In [138]:
# #read Data files
# data_train1 = pd.read_csv('my_final/train_none_Data.csv')
# data_test1 = pd.read_csv('my_final/test_none_Data.csv')

xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"max_depth"        : 6
    ,"subsample"        :0.8
    ,"colsample_bytree" :0.8
    ,"alpha"            :5
    ,"lambda"           :1
}
num_round = 30

d_train = xgb.DMatrix(X_train, y_train)
watchlist= [(d_train, "train")]
bst = xgb.train(params= xgb_params, dtrain=d_train, num_boost_round=num_round, evals=watchlist,verbose_eval = 10)

[00:46:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\gbm\gbtree.cc:138: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-logloss:0.51082
[10]	train-logloss:0.25091
[20]	train-logloss:0.24593
[29]	train-logloss:0.24491


## F1 optimization code to get the optimal f1 score

In [133]:
# Based on the paper Optimizing F-measure: A Tale of Two Approaches - https://arxiv.org/abs/1206.4625
# https://www.kaggle.com/mmueller/f1-score-expectation-maximization-in-o-n
"""
@author: Faron
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

'''
This kernel implements the O(n²) F1-Score expectation maximization algorithm presented in
"Ye, N., Chai, K., Lee, W., and Chieu, H.  Optimizing F-measures: A Tale of Two Approaches. In ICML, 2012."

It solves argmax_(0 <= k <= n,[[None]]) E[F1(P,k,[[None]])]
with [[None]] being the indicator for predicting label "None"
given posteriors P = [p_1, p_2, ... , p_n], where p_1 > p_2 > ... > p_n
under label independence assumption by means of dynamic programming in O(n²).
'''


class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)


### Merging the probablities for two models

In [140]:
# merging the probablities for two models and passing them to the f1 optimizer and saving the results in the file for submission

none_prob = pd.read_csv("my_final/None_Probablity.csv")
df_to_test = data_test1.copy()
df_to_test1 =df_to_test[['order_id','product_id']]
data_test = data_test1.drop(['Unnamed: 0', 'order_id','eval_set_x','eval_set_y','reordered','product_name'],axis=1)

preds = bst.predict(xgb.DMatrix(data_test))

df_to_test1['pred'] = preds
df_to_test = df_to_test1.copy()

df_to_test = df_to_test[['order_id','product_id','pred']].groupby('order_id').agg({'product_id': list, 'pred': list})

def sort_values(row):  
    ok = np.argsort(np.array(row['pred']))
    df_to_test.at[row.name,'product_id']=[row['product_id'][i] for i in ok][::-1]
    df_to_test.at[row.name,'pred'] = [row['pred'][i] for i in ok][::-1]

df_to_test.apply(sort_values,axis=1)

final_preds=[]
final_order_ids=[]
for index, row in tqdm(df_to_test.iterrows()):
    final_order_ids.append(index)
    opt=F1Optimizer.maximize_expectation(row.pred,none_prob[none_prob['order_id']==index]['none_pred'].values[0])
    best_k=opt[0]
    pred=df_to_test[df_to_test.index==index]['product_id'].values[0][:best_k]
    if(opt[1]):
        pred.append('None')
    final_preds.append(' '.join([str(v) for v in pred]))
submit=pd.DataFrame({'order_id': final_order_ids, 'products': final_preds})
submit=submit.sort_values('order_id')
submit.to_csv('my_final/sub.csv', index=False)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
75000it [55:26, 22.55it/s]


## Kaggle Submission result
<img src='my_final/GB_None_faron.PNG'>

## Pulic leader Board rank 526
<img src='my_final/public.PNG'>